In [1]:
import numpy as np
from scipy.linalg import expm

metrics for how close two matrices are to one another:

In [2]:
def trace_distance(rho1, rho2):

    diff = rho1 - rho2
    evals = np.linalg.eigvals(diff)
    
    return 0.5 * sum(np.abs(evals))

Single qubit operations:

In [3]:
X = np.array([[0, 1], [1, 0]])
Y = np.array([[0, -1j], [1j, 0]])
Z = np.array([[1, 0], [0, -1]])
I = np.array([[1, 0], [0, 1]])


def Ra(theta, phi):

    theta *= np.pi
    phi *= np.pi

    return np.array([[np.cos(theta / 2), -1j * np.exp(-1j * phi) * np.sin(theta / 2)], 
                     [-1j * np.exp(1j * phi) * np.sin(theta / 2), np.cos(theta / 2)]])

def Rx(theta):
    return Ra(theta, 0)

def Ry(theta):
    return Ra(theta, np.pi / 2)

def Rz(theta):

    theta *= np.pi

    return np.array([[np.exp(-1j * theta / 2), 0], [0, np.exp(1j * theta / 2)]])

In [4]:
def UR0(theta):

    theta *= np.pi

    exponent_matrix_1 = np.kron(np.kron(X, Z), Y)
    exponential_1 = np.exp(-1j * (theta / 2) * exponent_matrix_1)

    exponent_matrix_2 = np.kron(np.kron(Y, Z), X)
    exponential_2 = expm(-1j * (theta / 2) * exponent_matrix_2)

    return exponential_1 * exponential_2


def XX(theta):

    theta *= np.pi

    exponent_matrix = np.kron(X, X)
    exponential = expm(-1j * (theta / 2) * exponent_matrix)

    return exponential


def YX(theta):

    theta *= np.pi

    exponent_matrix = np.kron(Y, X)
    exponential = expm(-1j * (theta / 2) * exponent_matrix)

    return exponential


def XXX(theta):

    theta *= np.pi

    exponent_matrix = np.kron(np.kron(X, X), X)
    exponential = expm(-1j * (theta / 2) * exponent_matrix)

    return exponential


def XZY(theta):

    theta *= np.pi

    exponent_matrix = np.kron(np.kron(X, Z), Y)
    exponential = expm(-1j * (theta / 2) * exponent_matrix)

    return exponential 


def YZX(theta):

    theta *= np.pi

    exponent_matrix = np.kron(np.kron(Y, Z), X)
    exponential = expm(-1j * (theta / 2) * exponent_matrix)

    return exponential


def YXX(theta):

    theta *= np.pi

    exponent_matrix = np.kron(np.kron(Y, X), X)
    exponential = expm(-1j * (theta / 2) * exponent_matrix)

    return exponential

First verify that sandwiching an XX gate with Rz gates can obtain a YX gate

In [31]:
def YX_from_XX(theta_Ra, phi_Ra, theta_Rz, theta_XX):

    transformation_Ra = np.kron(Ra(theta_Ra, phi_Ra), I)
    transformation_Ra_dagger = np.kron(Ra(-1 * theta_Ra, phi_Ra), I)

    transformation_Rz = np.kron(Rz(theta_Rz), I)
    transformation_Rz_dagger = np.kron(Rz(-1 * theta_Rz), I)

    return transformation_Ra @ transformation_Rz @ XX(theta_XX) @ transformation_Rz_dagger @ transformation_Ra_dagger

In [32]:
print(f"TD: {trace_distance(YX_from_XX(0, 0, 0.5, 0.5), YX(0.5))}")
print(f"Max: ", max((YX_from_XX(0, 0, 0.5, 0.5) - YX(0.5)).flatten()))

TD: 2.220446049250313e-16
Max:  0j


Try getting YX gate of arbitrary angle from XX gate of fixed angle

In [33]:
def arbitrary_YX_from_XX(theta_Ra1, phi_Ra1, theta_Rz1, theta_Ra2, phi_Ra2, theta_Rz2, theta_XX):

    transformation_Ra1 = np.kron(Ra(theta_Ra1, phi_Ra1), I)
    transformation_Ra1_dagger = np.kron(Ra(-1 * theta_Ra1, phi_Ra1), I)

    transformation_Rz1 = np.kron(Rz(theta_Rz1), I)
    transformation_Rz1_dagger = np.kron(Rz(-1 * theta_Rz1), I)


    transformation_Ra2 = np.kron(I, Ra(theta_Ra2, phi_Ra2))
    transformation_Ra2_dagger = np.kron(I, Ra(-1 * theta_Ra2, phi_Ra2))

    transformation_Rz2 = np.kron(I, Rz(theta_Rz2))
    transformation_Rz2_dagger = np.kron(I, Rz(-1 * theta_Rz2))


    return transformation_Ra1 @ transformation_Rz1 @ transformation_Ra2 @ transformation_Rz2 @ XX(theta_XX) @ transformation_Rz1_dagger @ transformation_Ra1_dagger @ transformation_Rz2_dagger @ transformation_Ra2_dagger

In [40]:
theta_XX = 0.5

min_td = 1e5

target_theta_YX = 0.25

for theta_Ra1 in range(0, 100, 10):
    for phi_Ra1 in range(0, 200, 10):
        for theta_Rz1 in range(0, 200, 10):
            for theta_Ra2 in range(0, 100, 10):
                for phi_Ra2 in range(0, 200, 10):
                    for theta_Rz2 in range(0, 200, 10):

                        YX_from_combination = arbitrary_YX_from_XX(theta_Ra1 / 100, phi_Ra1 / 100, theta_Rz1 / 100, theta_Ra2 / 100, phi_Ra2 / 100, theta_Rz2 / 100, theta_XX)
                        YX_native = YX(target_theta_YX)
                        
                        td = trace_distance(YX_from_combination, YX_native)
                        
                        if td < min_td:
                            min_td = td
                        
                        print(f"theta_Ra1: {theta_Ra1 / 100}pi, phi_Ra1: {phi_Ra1 / 100}pi, theta_Rz1: {theta_Rz1 / 100}, theta_Ra2: {theta_Ra2 / 100}pi, phi_Ra2: {phi_Ra2 / 100}pi, theta_Rz2: {theta_Rz2 / 100}pi, td: {min_td}", end='\r')

KeyboardInterrupt: 

Try sandwhiching an XXX gate of fixed angle with an arbitrary single qubit gate on qubit 1 to get a YXX gate 

In [ ]:
def YXX_from_XXX(theta_Ra1, phi_Ra1, theta_Rz1, theta_XXX):

    transformation_Ra = np.kron(np.kron(Ra(theta_Ra, phi_Ra), I), I)
    transformation_Ra_dagger = np.kron(np.kron(Ra(-1 * theta_Ra, phi_Ra), I), I)

    transformation_Rz = np.kron(np.kron(Rz(theta_Rz), I), I)
    transformation_Rz_dagger = np.kron(np.kron(Rz(-1 * theta_Rz), I), I)

    return transformation_Ra @ transformation_Rz @ XXX(theta_XXX) @ transformation_Rz_dagger @ transformation_Ra_dagger

In [18]:
print(f"TD: {trace_distance(YXX_from_XXX(0, 0, 0.5, 0.5), YXX(0.5))}")
print(f"Max: ", max((YXX_from_XXX(0, 0, 0.5, 0.5) - YXX(0.5)).flatten()))

TD: 4.440892098500626e-16
Max:  0j


Fix the XXX angle and scan the rotation angles

In [28]:
theta_XXX = 0.5
theta_Rz = 0.5

min_td = 1e5

target_theta_YXX = 0.25

for theta_Ra in range(0, 100):
    for phi_Ra in range(0, 200):

        YXX_from_combination = YXX_from_XXX(theta_Ra / 100, phi_Ra / 100, theta_Rz, theta_XXX)
        YXX_native = YXX(target_theta_YXX)
        
        td = trace_distance(YXX_from_combination, YXX_native)
        
        if td < min_td:
            min_td = td
        
        print(f"theta_Ra: {theta_Ra / 100}pi, phi_Ra: {phi_Ra / 100}pi, td: {min_td}", end='\r')